<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import numpy as np
from numpy.random import RandomState
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score, roc_curve, roc_auc_score
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


В данных нет пропусков. Все типы данных определены верно

In [4]:
pd.set_option('display.max_colwidth', 1000)

In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [6]:
print('\nКоличество дубликатов в данных -', data.duplicated().sum())


Количество дубликатов в данных - 0


В данных нет дубликатов

In [7]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Присутствует сильный дисбаланс классов

</n>
Отчистим комментарий от лишних символов, приведем все к нижнему регисстру и к своей лемме.

In [8]:
data['text'] = data['text'].str.lower()

In [9]:
data = data.sample(50000).reset_index(drop=True)
def clear_text(string):
    new_string = re.sub(r"(?:\n|\r)", "", string)
    new_string = re.sub(r'[^a-zA-Z\ ]', " ", new_string).strip()
    return new_string
data['text'] = data['text'].apply(clear_text)

In [10]:
data.head()

,Unnamed: 0,text,toxic
0,45459,firstly please kindly specifically say what here is an inappropriate cut and paste afaik this was lastly moved with move button careful with reading the edit history please secondly on what do you base an erroneous claim that russian patronymic daughter of george as part of name is georgievna not yurievna georgievna rather is sort of semi anglic german western bastardized version whereas yurij is the translitered russian name on basis of which patronymics too should be please explain your credentials in assessing georgievna to be better than yurievna please give evidence that georgievna is used by modern high quality texts instead of yurievna georgievna may have been used in semi german anglic very old texts and in low quality texts afaik but never in today high quality works thirdly you apprently missed the point the of greece is the new element suggested above have you checked naming conventions of non reigning royals what is your reasoning for t...,0
1,27727,vandalyze my goodness did you bother to read this my first impression about you was that you were a super sensitive year old with a trigger happy finger after reading through your bio i came up with a slightly different view you didn t read or comprehend the importance of my suggestion i will continue pursuing this and if you wish to discuss it with me please contact me via my user page wangtopgun i offer full disclosure about myself my background various careers etc i suggest you read before you hit the delete key because you may be missing a gem all the best dean garner vandalyze i want this topic to be unprotected so i suggest you do so do you wanna go to war with me,0
2,32478,hi trpod i hear you are still being one of the horsemen of wikipedia trying to astroturf the narrative on the gamergate controversy page i congratulate you on your sterling efforts to maintain wikipedia s most uniquely biased article i hope you let us all know how much you are being paid at the end of this i hope you aren t doing it as a freebie kiss kiss,0
3,41349,utc i completely agree not only does the guy have penis problem but the figures are extremely anorexic aswell someone may also be partially motivated to promote exaggerated stereotypes the picture before with moderate figure had been used for long time and is good representation of average human figure talk talk contribs march,0
4,73613,you are the one attacking and libelling me i don t follow you around or make a nuisance on your talk page learn respect,0


In [11]:
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemm_list = [lemmatizer.lemmatize(w) for w in word_list]
    lemm_text = " ".join(lemm_list)
    return lemm_text
data['text'] = data['text'].apply(lemmatize)

<b> В данных нет пропусков, дубликатов, ошибок в типах данных. Текст в сообщениях подготовлен к обучения: приведен к нижнему регистру и своей лемме

## Обучение

In [13]:
features = data.drop(['toxic'], axis=1)
target = data['toxic']

In [14]:
features_train, features_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=12345)

Метод векторизации "TF-IDF"

In [15]:
corpus_train = features_train['text'].values
corpus_test = features_test['text'].values

In [16]:
count_tf_idf = TfidfVectorizer(ngram_range=(1,2),
                               stop_words=stop_words,
                               min_df=3,
                               max_df=0.9,
                               #strip_accents='unicode',
                               use_idf=1,
                               smooth_idf=1,
                               sublinear_tf=1)

In [17]:
tf_df = count_tf_idf.fit(corpus_train)

In [18]:
x_train = tf_df.transform(corpus_train)
x_test = tf_df.transform(corpus_test)

In [19]:
state = RandomState(12345)

<b>*Обучим модель LogisticRegression*

In [26]:
%%time

model_lr = LogisticRegression(class_weight='balanced', solver='liblinear', max_iter=1000)
params = dict(C=np.arange(1, 20, 1))
search_for_lr = RandomizedSearchCV(model_lr, 
                                   params,
                                   scoring='f1',
                                   random_state=state)
search_for_lr.fit(x_train, y_train)
print(search_for_lr.best_params_)
print(search_for_lr.best_score_)

{'C': 8}
0.7550633580488274
CPU times: user 3min 39s, sys: 3min 12s, total: 6min 52s
Wall time: 6min 52s


<b>*Обучим модель RandomForestClassifier*

In [21]:
%%time

model_rf = RandomForestClassifier(class_weight='balanced')
params = dict(n_estimators=np.arange(10, 100, 10),
              max_depth=np.arange(10, 100, 10),
              min_samples_leaf=[1, 3])
search_for_rf = RandomizedSearchCV(model_rf, 
                                   params,
                                   scoring='f1',
                                   random_state=state)
search_for_rf.fit(x_train, y_train)
print(search_for_rf.best_params_)
print(search_for_rf.best_score_)

{'n_estimators': 50, 'min_samples_leaf': 3, 'max_depth': 90}
0.5378645802415719
CPU times: user 4min 2s, sys: 0 ns, total: 4min 2s
Wall time: 4min 2s


<b>*Обучим модель LinearSVC*

In [30]:
%%time
model_lsvc = LinearSVC(random_state=state, class_weight='balanced')
params = dict(C=np.arange(0.1, 1, 0.1))
search_for_lsvc = RandomizedSearchCV(model_lsvc, 
                                   params,
                                   scoring='f1',
                                   random_state=state)
search_for_lsvc.fit(x_train, y_train)
print(search_for_lsvc.best_params_)
print(search_for_lsvc.best_score_)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'C': 0.7000000000000001}
0.7537884410727222
CPU times: user 13.2 s, sys: 0 ns, total: 13.2 s
Wall time: 13.2 s


<b> Вывод: было обучено 3 разных модели

## Тестирование лучшей модели

F1 выше у модели LogisticRegression, поэтому протестируем ее.

In [34]:
models = ['LogisticRegression', 'RandomForest', 'LinearSVC']
f1 = [search_for_lr.best_score_, search_for_rf.best_score_, search_for_lsvc.best_score_]

result = pd.DataFrame({
    'Модель': models, 
    'F1': f1
}) 
result

,Модель,F1
0,LogisticRegression,0.755063
1,RandomForest,0.537865
2,LinearSVC,0.753788


In [33]:
test_model_lr = LogisticRegression(solver='liblinear',
                                   C=9,
                                   class_weight='balanced',
                                   random_state=state,
                                   max_iter=1000)
test_model_lr.fit(x_train, y_train)
f1_test_lr = f1_score(y_test, test_model_lr.predict(x_test))
print(f1_test_lr)

0.7607407407407407


<b> Наилучшие показатели у модели "Логистическая регрессия". Результат на тестовой выборке: f1 = 0.76

## Выводы

<b> В этой работе я создавал модель, которая будет определять токсичные комментарии. Для этого все коментарии были приведены к лемме и нижнему регистру. Затем было обучено 3 модели LogisticRegression, RandomForest, LinearSVC. Логистическая регрессия показала точность выше всех, поэтому ее я проверили на тестовой выборке. F1>0,75 - значит поставленная цель выполнена.

<font color='green'><b>Полезные (и просто интересные) материалы:</b></font> \
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 

<font color='green'>Пример BERT с GPU:
```python
%%time
from tqdm import notebook
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings
        
features = np.concatenate(embeddings) 
```
Можно сделать предварительную проверку на наличие GPU.\
Например, так: ```device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")```\
Тогда вместо .cuda() нужно писать .to(device)

Если понравилась работа с текстами, то можешь посмотреть очень интересный (но очень-очень сложный) курс лекций: https://github.com/yandexdataschool/nlp_course .
</font>